In [9]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.Builder().appName('Netwoek-Analysis')\
.config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
.config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.12:5.3.1_for_spark_3") \
.getOrCreate()
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
# .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.12:5.3.2_for_spark_3") \
spark.conf.set('spark.sql.session.logLevel',"ERROR")

24/11/05 20:33:57 WARN Utils: Your hostname, codebase resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/05 20:33:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/de-ninja/.ivy2/cache
The jars for the packages stored in: /home/de-ninja/.ivy2/jars
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c0614be-2900-4d2c-a200-7e21d6a950c4;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.12;5.3.1_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.3.1 in central
	found org.neo4j.driver#neo4j-java-driver;4.4.17 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.apache.xbean#xbean-asm6-shaded;4.10 in central
	found org.neo4j#neo4j-cypher-dsl;2022.9.2 in central
	found org.apiguardian#apiguardian-api;1.1.2 in central
downloading https://repo1.maven.org/maven2/org/neo4j/neo4j-connector-apache-spark_2.12/5.3.1_for_spark_3/neo4j-connector-apache-spark_2.12-5.3.1_for_spark_3.jar ...
	[SUCCESSFUL ] org.neo4j#neo4j-connector-apache-spark_2.12;5.3.1_for_spark_3!neo4j-c

In [ ]:
data = spark.read\
.option('inferSchema',True)\
.option('header',True)\
.csv('transactions.csv')
data.createOrReplaceTempView('data')
data.cache() 
data.count() 

In [ ]:
# Create vertices DataFrame (distinct account IDs)
vertices_df = data.selectExpr("SENDER_ACCOUNT_ID as id").distinct() \
                             .union(data.selectExpr("RECEIVER_ACCOUNT_ID as id").distinct())

# Create edges DataFrame (transactions)
edges_df = data.selectExpr("SENDER_ACCOUNT_ID as src", "RECEIVER_ACCOUNT_ID as dst", "TX_AMOUNT as weight")

In [ ]:
from graphframes import GraphFrame

graph = GraphFrame(vertices_df, edges_df)

In [ ]:
# Write vertices to Neo4j
vertices_df.write \
    .format("neo4j") \
    .option("url", "bolt://localhost:7687") \
    .option("database", "neo4j") \
    .option("user", "neo4j") \
    .option("password", "DE-Ninja") \
    .option("batch.size", 1000) \
    .mode("overwrite") \
    .save()

# Write edges to Neo4j
edges_df.write \
    .format("neo4j") \
    .option("url", "bolt://localhost:7687") \
    .option("database", "neo4j") \
    .option("user", "neo4j") \
    .option("password", "DE-Ninja") \
    .option("batch.size", 1000) \
    .mode("overwrite") \
    .save()

In [8]:
%%time
import concurrent.futures
from neo4j import GraphDatabase

class Neo4jTransactionImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_transaction(self, tx_id, sender_account_id, receiver_account_id, tx_type, tx_amount, timestamp, is_fraud, alert_id):
        with self.driver.session() as session:
            session.write_transaction(self._create_and_link, tx_id, sender_account_id, receiver_account_id, tx_type, tx_amount, timestamp, is_fraud, alert_id)

    @staticmethod
    def _create_and_link(tx, tx_id, sender_account_id, receiver_account_id, tx_type, tx_amount, timestamp, is_fraud, alert_id):
        # Create or match the sender and receiver accounts
        tx.run("MERGE (s:Account {account_id: $sender_account_id}) "
                "MERGE (r:Account {account_id: $receiver_account_id}) "
                "CREATE (t:Transaction {tx_id: $tx_id, tx_type: $tx_type, tx_amount: $tx_amount, "
                "timestamp: $timestamp, is_fraud: $is_fraud, alert_id: $alert_id}) "
                "MERGE (s)-[:SENT]->(t) "
                "MERGE (t)-[:RECEIVED_BY]->(r)",
                sender_account_id=sender_account_id,
                receiver_account_id=receiver_account_id,
                tx_id=tx_id,
                tx_type=tx_type,
                tx_amount=tx_amount,
                timestamp=timestamp,
                is_fraud=is_fraud,
                alert_id=alert_id)

def insert_transaction(tx_data):
    uri = "neo4j://localhost:7687"  # Adjust as needed
    user = "neo4j"
    password = "DE-Ninja"  # Replace with your Neo4j password
    importer = Neo4jTransactionImporter(uri, user, password)
    
    # Unpack the transaction data
    importer.create_transaction(*tx_data)
    importer.close()

if __name__ == "__main__":
    # Example data
    transactions_df = data.select("*").collect()  # Assuming `data` is your Spark DataFrame
    transactions_list = [tuple(row) for row in transactions_df]

    # Specify the number of processes
    num_processes = 10  # Adjust this value based on your system and data size

    # Insert transactions in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
        executor.map(insert_transaction, transactions_list)

<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
<timed exec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
 Wait list:ExclusiveLock[
Client[242682] waits for [ForsetiClient[transactionId=242882, clientId=350]]])
 Wait

KeyboardInterrupt: 

 Wait list:ExclusiveLock[
Client[243572] waits for [ForsetiClient[transactionId=243778, clientId=355]]])
 Wait list:ExclusiveLock[
Client[243572] waits for [ForsetiClient[transactionId=243685, clientId=360]]])
 Wait list:ExclusiveLock[
Client[243822] waits for [ForsetiClient[transactionId=243844, clientId=344]]])


In [7]:
import concurrent.futures
from neo4j import GraphDatabase
from math import ceil

class Neo4jTransactionImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_transactions_batch(self, transactions_batch):
        with self.driver.session() as session:
            session.write_transaction(self._create_and_link_batch, transactions_batch)

    @staticmethod
    def _create_and_link_batch(tx, transactions_batch):
        for tx_data in transactions_batch:
            tx_id, sender_account_id, receiver_account_id, tx_type, tx_amount, timestamp, is_fraud, alert_id = tx_data
            tx.run("MERGE (s:Account {account_id: $sender_account_id}) "
                   "MERGE (r:Account {account_id: $receiver_account_id}) "
                   "CREATE (t:Transaction {tx_id: $tx_id, tx_type: $tx_type, tx_amount: $tx_amount, "
                   "timestamp: $timestamp, is_fraud: $is_fraud, alert_id: $alert_id}) "
                   "MERGE (s)-[:SENT]->(t) "
                   "MERGE (t)-[:RECEIVED_BY]->(r)",
                   sender_account_id=sender_account_id,
                   receiver_account_id=receiver_account_id,
                   tx_id=tx_id,
                   tx_type=tx_type,
                   tx_amount=tx_amount,
                   timestamp=timestamp,
                   is_fraud=is_fraud,
                   alert_id=alert_id)

def insert_transactions_batch(batch):
    uri = "neo4j://localhost:7687"  # Adjust as needed
    user = "neo4j"
    password = "DE-Ninja"  # Replace with your Neo4j password
    importer = Neo4jTransactionImporter(uri, user, password)
    
    # Insert the batch of transactions
    importer.create_transactions_batch(batch)
    importer.close()

def batch_data(data_list, batch_size):
    """Splits the data into batches."""
    for i in range(0, len(data_list), batch_size):
        yield data_list[i:i + batch_size]

if __name__ == "__main__":
    # Example data from Spark DataFrame
    transactions_df = data.select("*").collect()  # Assuming `data` is your Spark DataFrame
    transactions_list = [tuple(row) for row in transactions_df]

    # Define batch size
    batch_size = 10000  # Adjust batch size as needed

    # Split data into batches
    transaction_batches = list(batch_data(transactions_list, batch_size))

    # Specify the number of processes
    num_processes = 10  # Adjust this value based on your system and data size

    # Insert transactions in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
        executor.map(insert_transactions_batch, transaction_batches)


/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link_batch, transactions_batch)
/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link_batch, transactions_batch)
/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link_batch, transactions_batch)
/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link_batch, transactions_batch)
/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link_batch, transactions_batch)
/tmp/ipykernel_65340/505958162.py:14: DeprecationWarning: write_t

KeyboardInterrupt: 

 Wait list:ExclusiveLock[
Client[242194] waits for [ForsetiClient[transactionId=242193, clientId=347]]])
 Wait list:ExclusiveLock[
Client[242195] waits for [ForsetiClient[transactionId=242197, clientId=356],ForsetiClient[transactionId=242188, clientId=357],ForsetiClient[transactionId=242192, clientId=332]]])
 Wait list:ExclusiveLock[
Client[242188] waits for [ForsetiClient[transactionId=242195, clientId=344],ForsetiClient[transactionId=242194, clientId=345],ForsetiClient[transactionId=242192, clientId=332],ForsetiClient[transactionId=242189, clientId=335]]])
 Wait list:ExclusiveLock[
Client[242189] waits for [ForsetiClient[transactionId=242188, clientId=357],ForsetiClient[transactionId=242195, clientId=344],ForsetiClient[transactionId=242194, clientId=345],ForsetiClient[transactionId=242192, clientId=332]]])
 Wait list:ExclusiveLock[
Client[242189] waits for [ForsetiClient[transactionId=242194, clientId=345]]])
 Wait list:ExclusiveLock[
Client[242189] waits for [ForsetiClient[transacti